In [1]:
pip install requests bs4 fake_useragent

In [ ]:
%%shell
# https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

apt-get update
apt-get install chromium chromium-driver

pip install selenium

In [2]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import time
import random
import string
import re

def scrape(url):
    """ Fonction qui retourne la page recherché sous BeautifulSoup """

    headers = {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': UserAgent().random,
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.fr/',
        "Accept-Language": "fr-FR,fr;q=0.9",
        'Cache-Control': 'max-age=0',
    }

    r = requests.get(url, headers=headers)

    if "To discuss automated access to Amazon data please contact api-services-support@amazon.com." in r.text:
      print("Amazon nous a bloqué, on ressaie dans 5s")
      time.sleep(5)
      return scrape(url)


    if r.status_code > 500:
            print(f"La page {url} a été bloqué par Amazon avec le code suivant : {r.status_code} on ressaie dans 5s")
            time.sleep(5)
            return scrape(url)

    return BeautifulSoup(r.text, 'html.parser')



def search_result(source_code):

    """ Fonction qui récupère l'id, le titre, le prix , l'image et le lien du produit avec BS4 """

    donnee_produits = {}
    produits = source_code.find_all('div', {'data-asin': True})
    for produit in produits:
        asin = produit['data-asin']
        titre = produit.select_one('.s-line-clamp-4 a')
        prix = produit.select_one('.a-offscreen')
        image = produit.find('img', {'class': 's-image'})
        lien = f"https://www.amazon.fr/dp/{asin}"
        if asin not in donnee_produits:
            donnee_produits[asin] = [titre.text, prix.text,image['src'], lien]
    return donnee_produits

def get_proxies():
    """ Fonction de retour de proxy """
    username = "n7ndqhb1yncd2ke"
    password = "0iofswgh63zh6d8"
    proxy = "rp.proxyscrape.com:6060"
    proxy_auth = "{}:{}@{}".format(username, password, proxy)
    proxies = {
    "http":"http://{}".format(proxy_auth)
    }

    return proxies

def generate_session_id(length=10):
    characters = string.ascii_letters + string.digits
    session_id = ''.join(random.choice(characters) for i in range(length))
    return session_id

def send_request(page, asin, session):
    headers = {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': UserAgent().random,
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.fr/',
        "Accept-Language": "fr-FR,fr;q=0.9",
        'Cache-Control': 'max-age=0',
    }

    requete = {
        "reviewerType": "all_reviews",
        "formatType": "all_formats",
        "pageNumber": page,
        "deviceType": "desktop",
        "canShowIntHeader": "undefined",
        "reftag": f"cm_cr_arp_d_paging_btm_next_{page}",
        "pageSize": 10,
        "asin": asin,
        "scope": "reviewsAjax0"
    }

    cookies = {
        "session-id": generate_session_id(),
    }

    r = session.post(
        f"https://www.amazon.fr/hz/reviews-render/ajax/reviews/get/ref=cm_cr_arp_d_paging_btm_next_{page}",
        data=requete,
        headers=headers,
        cookies=cookies
    )
    return r




In [3]:
def check_connexion(asin,page,source_code,session):

      try:

        if "Commentaires en ligne:" in source_code.find_all('title')[0].text:

          print("Récupération de la page réussie")

          return source_code

      except:pass

      print("Y'a eu un beug on réessaie.")

      print(source_code)

      r =send_request(page,asin,session)

      return source_code


def repl_func(match):
    if match == True:
        return ""

def get_review(asin,session,page=1,review=[]):

    source_code = send_request(page,asin,session)

    if not source_code.ok:
      return get_review(asin,session,page)

    source_code = BeautifulSoup(source_code.text)
    avis = source_code.find_all('span',{'data-hook':'\\"review-body\\"'})
    star = source_code.find_all('i',{'data-hook':'\\"review-star-rating\\"'})
    for ligne1,ligne2 in zip(avis,star):
      review.append((re.sub(r"\\n", repl_func, ligne1.text),ligne2.text[0]))

    if not avis:
      return review
    return get_review(asin,session,page+1)


with requests.Session() as session:
  review = get_review("B09G9HWQYT",session)


In [4]:
print(review)

[("        En tant qu'utilisateur régulier d'iPhone, je dois dire que le modèle iPhone 13 blanc est une véritable œuvre d'art. Sa conception élégante et épurée est à couper le souffle. La couleur blanche apporte une touche de sophistication et de pureté qui se démarque vraiment.En ce qui concerne les performances, l'iPhone 13 ne déçoit pas. Son processeur ultra-rapide et ses fonctionnalités avancées en font un compagnon fiable pour toutes mes tâches quotidiennes, qu'il s'agisse de répondre aux e-mails, de naviguer sur Internet ou de capturer des moments précieux avec sa caméra de qualité professionnelle.Ce que j'apprécie particulièrement, c'est la durée de vie de la batterie améliorée. Je n'ai plus à m'inquiéter de manquer de batterie au milieu de la journée, ce qui me permet de rester connecté plus longtemps sans avoir à chercher une prise de courant.De plus, l'écran Super Retina XDR offre des couleurs éclatantes et une netteté incroyable, ce qui rend chaque expérience visuelle captiv

In [ ]:
import uuid

def generate_uuid_cookie():
    return str(uuid.uuid4())

uuid_guest = generate_uuid_cookie()

cookies = {
    "uuid_guest":uuid_guest,
    "uuid_guest_backup":uuid_guest,
    "youchat_personalization":True,
    "youchat_smart_learn":True,
    "no_activation_trial_event_sent":True,
    "has_seen_trial_limit_reached_upsell":True,
    "should_notify_trial_limit_reached":False,
    "eg4":False,
    "chat_mode":"default",
    "youpro_subscription":False,
    "you_subscription":"free",
    "region":"fr-FR",
    "safesearch_guest":"Off",
    "afUserId":generate_uuid_cookie()
    }
request = {
    "q":"Peux tu seulement me répondre par des -1 ou 1, je vais t'envoyer une description de produit et 1 avis et tu me dis s'il est conforme ou pas a la description c'est à dire si il valide ou pas. (réponds moi par 0 si tu as compris sans faire de phrase)",
    "fromSearchBar":"true",
    "tbm":"youchat"
}
headers = {
    "Accept-Ch":"Sec-CH-UA-Bitness, Sec-CH-UA-Arch, Sec-CH-UA-Full-Version, Sec-CH-UA-Mobile, Sec-CH-UA-Model, Sec-CH-UA-Platform-Version, Sec-CH-UA-Platform, Sec-CH-UA, UA-Bitness, UA-Arch, UA-Full-Version, UA-Mobile, UA-Model, UA-Platform-Version, UA-Platform, UA",
    "Cache-Control":"private, no-cache, max-age=0, must-revalidate",

}

In [6]:
mon_shop = {
"ordi_portable" : "https://www.amazon.fr/s?rh=n%3A429879031&fs=true&ref=lp_429879031_sar",
"ecran" : "https://www.amazon.fr/b/?node=427938031&ref_=Oct_d_odnav_d_340858031_1&pd_rd_w=OBD9j&content-id=amzn1.sym.16e687e1-cc57-4a8e-bc86-97edeb38eb87&pf_rd_p=16e687e1-cc57-4a8e-bc86-97edeb38eb87&pf_rd_r=P5GS25WEMEM8SSZ640BV&pd_rd_wg=lZYse&pd_rd_r=b651bd98-17c0-4026-82e4-895d017e3873",
"tablettes" : "https://www.amazon.fr/s?rh=n%3A429882031&fs=true&ref=lp_429882031_sar",
"accessoire" : "https://www.amazon.fr/s?rh=n%3A427942031&fs=true&ref=lp_427942031_sar",
"stockage":"https://www.amazon.fr/s?rh=n%3A427940031&fs=true&ref=lp_427940031_sar",
"composants":"https://www.amazon.fr/s?rh=n%3A427941031&fs=true&ref=lp_427941031_sar",
"ordinateur_bureau":"https://www.amazon.fr/s?rh=n%3A427937031&fs=true&ref=lp_427937031_sar",
"pc_portable" : "https://www.amazon.fr/s?i=specialty-aps&rh=n%3A21734135031&fs=true&ref=lp_21734135031_sar",
"pc_fixe" : "https://www.amazon.fr/s?i=specialty-aps&rh=n%3A21734134031&fs=true&ref=lp_21734134031_sar",
"ecran" : "https://www.amazon.fr/s?i=specialty-aps&rh=n%3A21734133031&fs=true&ref=lp_21734133031_sar",
"composants" : "https://www.amazon.fr/s?i=specialty-aps&rh=n%3A21734136031&fs=true&ref=lp_21734136031_sar",
"routeur" : "https://www.amazon.fr/s?i=specialty-aps&rh=n%3A21734132031&fs=true&ref=lp_21734132031_sar",
"jeux_pc" : "https://www.amazon.fr/s?i=specialty-aps&srs=21734154031&rh=n%3A21734154031&fs=true&ref=lp_21734154031_sar",
"chaises" : "https://www.amazon.fr/s?rh=n%3A92566381031&fs=true&ref=lp_92566381031_sar",
"bureau" : "https://www.amazon.fr/s?rh=n%3A92566380031&fs=true&ref=lp_92566380031_sar",
"ps5" : "https://www.amazon.fr/s?rh=n%3A20904296031",
"xbox_series_x" : "https://www.amazon.fr/s?rh=n%3A20904313031",
"nintendo_switch" : "https://www.amazon.fr/b/ref=GSNavConsole_3?pf_rd_r=V8V38J7P8FC5V02Z31KX&pf_rd_p=4e64089c-8cfd-41fb-bce3-9ef7489a16c9&pf_rd_m=A1X6FK5RDHNB96&pf_rd_s=merchandised-search-5&pf_rd_t=&pf_rd_i=21734155031&ie=UTF8&node=12366241031",
"tv_home_et_cinema" : "https://www.amazon.fr/s?rh=n%3A22523258031&fs=true&ref=lp_22523258031_sar",
"jeux_ps5" : "https://www.amazon.fr/s?rh=n%3A20904297031",
"jeux_xbox_series_x" : "https://www.amazon.fr/s?rh=n%3A20904314031",
"jeux_nintendo_switch" : "https://www.amazon.fr/b/ref=GSNavConsole_7?pf_rd_r=V8V38J7P8FC5V02Z31KX&pf_rd_p=4e64089c-8cfd-41fb-bce3-9ef7489a16c9&pf_rd_m=A1X6FK5RDHNB96&pf_rd_s=merchandised-search-5&pf_rd_t=&pf_rd_i=21734155031&node=12366242031",
"store_e_sport" : "https://www.amazon.fr/s?i=specialty-aps&srs=21734129031&rh=n%3A21734129031&fs=true&ref=lp_21734129031_sar",
"casque_realite_virtuelle" : "https://www.amazon.fr/s?rh=n%3A22473144031&fs=true&ref=lp_22473144031_sar",
"telephone_portable_pour_les_jeux" : "https://www.amazon.fr/s?i=specialty-aps&srs=21734131031&rh=n%3A21734131031&fs=true&ref=lp_21734131031_sar",
"produits_derive" : "https://www.amazon.fr/s?k=gaming+merchandise&pf_rd_i=21734155031&pf_rd_m=A1X6FK5RDHNB96&pf_rd_p=75c799a9-3fba-4b1e-bce0-df70c3f451d3&pf_rd_r=V8V38J7P8FC5V02Z31KX&pf_rd_s=merchandised-search-6&ref=GSMoreGaming_4",
"jeux_retro" : "https://www.amazon.fr/s?rh=n%3A26864502031&fs=true&ref=lp_26864502031_sar",
"carte_prepayees" : "https://www.amazon.fr/s?rh=n%3A14135877031&fs=true&ref=lp_14135877031_sar",
"snacks_et_boissons" : "https://www.amazon.fr/s?rh=n%3A27861662031&fs=true&ref=lp_27861662031_sar",
"tous_les_accessoires" : "https://www.amazon.fr/s?rh=n%3A92555569031&fs=true&ref=lp_92555569031_sar",
"accessoires_xbox" : "https://www.amazon.fr/s?rh=n%3A20904298031&fs=true&ref=lp_20904298031_sar",
"accessoires_playstation" : "https://www.amazon.fr/s?rh=n%3A20904281031&fs=true&ref=lp_20904281031_sar",
"accessoires_nintendo_switch" : "https://www.amazon.fr/s?rh=n%3A12366240031&fs=true&ref=lp_12366240031_sar",
"accessoires_pour_pc" : "https://www.amazon.fr/s?rh=n%3A93369491031&fs=true&ref=lp_93369491031_sar",
"claviers" : "https://www.amazon.fr/s?i=specialty-aps&rh=n%3A21734146031&fs=true&ref=lp_21734146031_sar",
"souris" : "https://www.amazon.fr/s?i=specialty-aps&rh=n%3A21734152031&fs=true&ref=lp_21734152031_sar",
"serveurs" : "https://www.amazon.fr/b/?node=429881031&ref_=Oct_d_odnav_d_340858031_6&pd_rd_w=bJlYD&content-id=amzn1.sym.e2663722-0d50-47c5-a7be-a70aba956841&pf_rd_p=e2663722-0d50-47c5-a7be-a70aba956841&pf_rd_r=8F6BZV31J4AC881TFJQK&pd_rd_wg=ixKec&pd_rd_r=dc0b0eed-d71d-40ca-aa14-d403b23e665d",
"casque" : "https://www.amazon.fr/s?i=specialty-aps&srs=21734147031&rh=n%3A21734147031&fs=true&ref=lp_21734147031_sar",
"reseaux" : "https://www.amazon.fr/s?rh=n%3A427939031&fs=true&ref=lp_427939031_sar",
"high_tech" : "https://www.amazon.fr/s?rh=n%3A13921051&fs=true&ref=lp_13921051_sar"
}

In [9]:
for categorie in mon_shop:
  print(mon_shop[categorie])

https://www.amazon.fr/s?rh=n%3A429879031&fs=true&ref=lp_429879031_sar
https://www.amazon.fr/s?i=specialty-aps&rh=n%3A21734133031&fs=true&ref=lp_21734133031_sar
https://www.amazon.fr/s?rh=n%3A429882031&fs=true&ref=lp_429882031_sar
https://www.amazon.fr/s?rh=n%3A427942031&fs=true&ref=lp_427942031_sar
https://www.amazon.fr/s?rh=n%3A427940031&fs=true&ref=lp_427940031_sar
https://www.amazon.fr/s?i=specialty-aps&rh=n%3A21734136031&fs=true&ref=lp_21734136031_sar
https://www.amazon.fr/s?rh=n%3A427937031&fs=true&ref=lp_427937031_sar
https://www.amazon.fr/s?i=specialty-aps&rh=n%3A21734135031&fs=true&ref=lp_21734135031_sar
https://www.amazon.fr/s?i=specialty-aps&rh=n%3A21734134031&fs=true&ref=lp_21734134031_sar
https://www.amazon.fr/s?i=specialty-aps&rh=n%3A21734132031&fs=true&ref=lp_21734132031_sar
https://www.amazon.fr/s?i=specialty-aps&srs=21734154031&rh=n%3A21734154031&fs=true&ref=lp_21734154031_sar
https://www.amazon.fr/s?rh=n%3A92566381031&fs=true&ref=lp_92566381031_sar
https://www.amazon.f

In [ ]:
def quatres_etoiles(lien):
  source_code = scrape(lien)
  href = source_code.find_all("li",{"id":"p_72/437873031"})
  if not href:
      href = source_code.find_all("li",{"id":"p_72/184909031"})
      if not href:
        return quatres_etoiles(lien)
  return "https://www.amazon.fr"+href[0].select_one('a')['href']


In [ ]:
href = quatres_etoiles("https://www.amazon.fr/s?rh=n%3A427939031&fs=true&ref=lp_427939031_sar")

In [ ]:
for categorie in mon_shop:
  href = quatres_etoiles(mon_shop[categorie])
  mon_shop[categorie] = href
print(mon_shop)

Amazon nous a bloqué, on ressaie dans 5s
Amazon nous a bloqué, on ressaie dans 5s
Amazon nous a bloqué, on ressaie dans 5s
Amazon nous a bloqué, on ressaie dans 5s
Amazon nous a bloqué, on ressaie dans 5s
Amazon nous a bloqué, on ressaie dans 5s
Amazon nous a bloqué, on ressaie dans 5s
Amazon nous a bloqué, on ressaie dans 5s
{'ordi_portable': 'https://www.amazon.fr/s?i=computers&rh=n%3A429879031%2Cp_72%3A437873031&dc&fs=true&qid=1705003247&rnid=437872031&ref=sr_nr_p_72_1&ds=v1%3AZjUdxXdQvb2Nq91AfqrfymHwIm3%2BGotg3kJTB1IxlCQ', 'ecran': 'https://www.amazon.fr/s?i=specialty-aps&rh=n%3A21734133031%2Cp_72%3A437873031&dc&fs=true&qid=1705003254&rnid=437872031&ref=sr_nr_p_72_1&ds=v1%3AvnKhp6DnaVRdCUou%2BP0VZxjkRqyIAGXy%2B9ZyJoFxmuA', 'tablettes': 'https://www.amazon.fr/s?i=computers&rh=n%3A429882031%2Cp_72%3A437873031&dc&fs=true&qid=1705003255&rnid=437872031&ref=sr_nr_p_72_1&ds=v1%3AAAtXGEKVNYLOWrN3n%2BsV1JVHK3IGRXmiTagBAm5LDag', 'accessoire': 'https://www.amazon.fr/s?i=computers&rh=n%3A42794

In [ ]:
mon_shop ={'ordi_portable': 'https://www.amazon.fr/s?i=computers&rh=n%3A429879031%2Cp_72%3A437873031&dc&fs=true&qid=1705003247&rnid=437872031&ref=sr_nr_p_72_1&ds=v1%3AZjUdxXdQvb2Nq91AfqrfymHwIm3%2BGotg3kJTB1IxlCQ', 'ecran': 'https://www.amazon.fr/s?i=specialty-aps&rh=n%3A21734133031%2Cp_72%3A437873031&dc&fs=true&qid=1705003254&rnid=437872031&ref=sr_nr_p_72_1&ds=v1%3AvnKhp6DnaVRdCUou%2BP0VZxjkRqyIAGXy%2B9ZyJoFxmuA', 'tablettes': 'https://www.amazon.fr/s?i=computers&rh=n%3A429882031%2Cp_72%3A437873031&dc&fs=true&qid=1705003255&rnid=437872031&ref=sr_nr_p_72_1&ds=v1%3AAAtXGEKVNYLOWrN3n%2BsV1JVHK3IGRXmiTagBAm5LDag', 'accessoire': 'https://www.amazon.fr/s?i=computers&rh=n%3A427942031%2Cp_72%3A437873031&dc&fs=true&qid=1705003257&rnid=437872031&ref=sr_nr_p_72_1&ds=v1%3A%2FzCDt4mP%2BoNE9DDAC%2FAvwX6XSq%2F%2BDcQ%2BVfnk5xyKZnQ', 'stockage': 'https://www.amazon.fr/s?i=computers&rh=n%3A427940031%2Cp_72%3A437873031&dc&fs=true&qid=1705003258&rnid=437872031&ref=sr_nr_p_72_1&ds=v1%3Ac%2BX%2F88QmtXyWy1G%2B%2BvDNYE%2FhyFduhjz7E27lj37rJWk', 'composants': 'https://www.amazon.fr/s?i=computers&rh=n%3A21734136031%2Cp_72%3A437873031&dc&fs=true&qid=1705003265&rnid=437872031&ref=sr_nr_p_72_1&ds=v1%3AzMng4rba8giyM4XLMGJJOedjAxEfX64EtgwYhckps7o', 'ordinateur_bureau': 'https://www.amazon.fr/s?i=computers&rh=n%3A427937031%2Cp_72%3A437873031&dc&fs=true&qid=1705003272&rnid=437872031&ref=sr_nr_p_72_1&ds=v1%3Alx2QEt33J1R4xADUNEG51dgM01YLukCE650d0vcb%2FN8', 'pc_portable': 'https://www.amazon.fr/s?i=specialty-aps&rh=n%3A21734135031%2Cp_72%3A437873031&dc&fs=true&qid=1705003278&rnid=437872031&ref=sr_nr_p_72_1&ds=v1%3AlNWobLYTwu40DpJPyMqbzQLpRA1nH4L0IhMMYkVOhdI', 'pc_fixe': 'https://www.amazon.fr/s?i=specialty-aps&rh=n%3A21734134031%2Cp_72%3A437873031&dc&fs=true&qid=1705003282&rnid=437872031&ref=sr_nr_p_72_1&ds=v1%3AmG66cHwaK7zFVBivaQUzJFsvQXYn2Aap9HW8yd71xH0', 'routeur': 'https://www.amazon.fr/s?i=specialty-aps&rh=n%3A21734132031%2Cp_72%3A437873031&dc&fs=true&qid=1705003283&rnid=437872031&ref=sr_nr_p_72_1&ds=v1%3AfI8NKzYMcQpORRyq0SjMdtea9AbYODNN70jCoESsyKw', 'jeux_pc': 'https://www.amazon.fr/s?i=specialty-aps&srs=21734154031&rh=n%3A21734154031%2Cp_72%3A437873031&dc&fs=true&qid=1705003284&rnid=437872031&ref=sr_nr_p_72_1&ds=v1%3AgOPUxTB2zuoY42yitZai9zcqo6xOziNY5eVSXF94Eeo', 'chaises': 'https://www.amazon.fr/s?i=kitchen&rh=n%3A92566381031%2Cp_72%3A437873031&dc&fs=true&qid=1705003286&rnid=437872031&ref=sr_nr_p_72_1&ds=v1%3A4T0nIqAgHF1w28hfXO%2BTQiVfzcLP4dIH0wIGxwVXB7Y', 'bureau': 'https://www.amazon.fr/s?i=kitchen&rh=n%3A92566380031%2Cp_72%3A437873031&dc&fs=true&qid=1705003292&rnid=437872031&ref=sr_nr_p_72_1&ds=v1%3AdFMiuxcQ4w0GgmezIcGyL%2F6ne1lSz7RNHYe52Qu5oac', 'ps5': 'https://www.amazon.fr/s?i=videogames&rh=n%3A20904296031%2Cp_72%3A184909031&dc&qid=1705003299&rnid=184896031&ref=sr_nr_p_72_1&ds=v1%3A%2FmyRYz7rFNko9l5Fo%2F9wC8EUkxzn1NyW1xyPVBXGGiY', 'xbox_series_x': 'https://www.amazon.fr/s?i=videogames&rh=n%3A20904313031%2Cp_72%3A184909031&dc&qid=1705003300&rnid=184896031&ref=sr_nr_p_72_1&ds=v1%3AVWHYTgqR%2FZwENcZEbO%2BeaJ%2FbIpBVysB97xmxA26RsPU', 'nintendo_switch': 'https://www.amazon.fr/s?keywords=Consoles+pour+Nintendo+Switch&i=videogames&rh=n%3A12366241031%2Cp_72%3A184909031&dc&c=ts&qid=1705003301&rnid=184896031&ts_id=12366241031&ref=sr_nr_p_72_1&ds=v1%3AS3V1r3PfwxX%2F1TCRIIFUIwNtopom7mHaTJUt%2FAeT7vc', 'tv_home_et_cinema': 'https://www.amazon.fr/s?i=electronics&rh=n%3A22523258031%2Cp_72%3A437873031&dc&fs=true&qid=1705003302&rnid=437872031&ref=sr_nr_p_72_1&ds=v1%3ABiT%2FY3vCE8L0poOCfKykc8WP5K7VDCpokoDG27CR2as', 'jeux_ps5': 'https://www.amazon.fr/s?i=videogames&rh=n%3A20904297031%2Cp_72%3A184909031&dc&qid=1705003303&rnid=184896031&ref=sr_nr_p_72_1&ds=v1%3AedeqQ%2BfB1X6bE9j2YKXV2Z2rBv6QnUSXdb5DGQBGlZk', 'jeux_xbox_series_x': 'https://www.amazon.fr/s?i=videogames&rh=n%3A20904314031%2Cp_72%3A184909031&dc&qid=1705003305&rnid=184896031&ref=sr_nr_p_72_1&ds=v1%3AycoMSHPELi86sf7Bgow0YZe8O9Q2fluzXJUrQA74eUA', 'jeux_nintendo_switch': 'https://www.amazon.fr/s?keywords=Jeux+pour+Nintendo+Switch&i=videogames&rh=n%3A12366242031%2Cp_72%3A184909031&dc&c=ts&qid=1705003306&rnid=184896031&ts_id=12366242031&ref=sr_nr_p_72_1&ds=v1%3AHiuVWoSOrElJhKrZ60rrXqUy6azxDT1vGZab0JCU7Ck', 'store_e_sport': 'https://www.amazon.fr/s?i=specialty-aps&srs=21734129031&rh=n%3A21734129031%2Cp_72%3A437873031&dc&fs=true&qid=1705003307&rnid=437872031&ref=sr_nr_p_72_1&ds=v1%3AGgxji7xnto6qfGMYGGYXyctMXvCnA0A3GQIWWDjrbg4', 'casque_realite_virtuelle': 'https://www.amazon.fr/s?i=videogames&rh=n%3A22473144031%2Cp_72%3A184909031&dc&fs=true&qid=1705003309&rnid=184896031&ref=sr_nr_p_72_1&ds=v1%3A6ruTVmJzxORDZdtY%2Flz%2BvlWUxYVnySz0O6MuAOhX4Wg', 'telephone_portable_pour_les_jeux': 'https://www.amazon.fr/s?i=specialty-aps&srs=21734131031&rh=n%3A21734131031%2Cp_72%3A437873031&dc&fs=true&qid=1705003316&rnid=437872031&ref=sr_nr_p_72_1&ds=v1%3AMqrScglw5gQf%2FPd8ulQApuFlS%2FZfJWefUCGWkTkNY%2F4', 'produits_derive': 'https://www.amazon.fr/s?k=gaming+merchandise&rh=n%3A530490%2Cp_72%3A184909031&dc&pf_rd_i=21734155031&pf_rd_m=A1X6FK5RDHNB96&pf_rd_p=75c799a9-3fba-4b1e-bce0-df70c3f451d3&pf_rd_r=V8V38J7P8FC5V02Z31KX&pf_rd_s=merchandised-search-6&qid=1705003317&rnid=184896031&ref=sr_nr_p_72_1&ds=v1%3AjuzY6AsxcdQC%2B%2B3t1RDYlr5WJAWb2FtQJFDYSlA8zlE', 'jeux_retro': 'https://www.amazon.fr/s?i=videogames&rh=n%3A26864502031%2Cp_72%3A184909031&dc&fs=true&qid=1705003318&rnid=184896031&ref=sr_nr_p_72_1&ds=v1%3A0kfHvOSWvxiuGaUpYXHY3pXSjbpx%2BFhCNCcU48h0VuA', 'carte_prepayees': 'https://www.amazon.fr/s?i=videogames&rh=n%3A14135877031%2Cp_72%3A184909031&dc&fs=true&qid=1705003320&rnid=184896031&ref=sr_nr_p_72_1&ds=v1%3AiKQH4lgk3EdebkMUmsRQfXQ7og1%2FD%2Blk7CuDNOAHipE', 'snacks_et_boissons': 'https://www.amazon.fr/s?i=grocery&rh=n%3A27861662031%2Cp_72%3A437873031&dc&fs=true&qid=1705003321&rnid=437872031&ref=sr_nr_p_72_1&ds=v1%3AOibtqZudmWIGRTTLhYYP3ZvnPjk1xbdCH30bd9l9u6A', 'tous_les_accessoires': 'https://www.amazon.fr/s?i=videogames&rh=n%3A92555569031%2Cp_72%3A184909031&dc&fs=true&qid=1705003322&rnid=184896031&ref=sr_nr_p_72_1&ds=v1%3AzG2QuAd6015259LtHSF6geaUP1YTsUxqSGgKtraCMj8', 'accessoires_xbox': 'https://www.amazon.fr/s?i=videogames&rh=n%3A20904298031%2Cp_72%3A184909031&dc&fs=true&qid=1705003323&rnid=184896031&ref=sr_nr_p_72_1&ds=v1%3AShRfy4apxHbp1Av5dA3wVFfPky6pf%2FqbcjSdVpJXdYc', 'accessoires_playstation': 'https://www.amazon.fr/s?i=videogames&rh=n%3A20904281031%2Cp_72%3A184909031&dc&fs=true&qid=1705003324&rnid=184896031&ref=sr_nr_p_72_1&ds=v1%3Aes2LYfmGgRTbF%2FKAQnYD16cmHR9l2%2FZI%2FJl3ZsQd0vA', 'accessoires_nintendo_switch': 'https://www.amazon.fr/s?i=videogames&rh=n%3A12366240031%2Cp_72%3A184909031&dc&fs=true&qid=1705003325&rnid=184896031&ref=sr_nr_p_72_1&ds=v1%3AGcSdO6h%2BEa4Y9HNzbtEFyiob5Erf1rgOyW%2F5ykKHtNA', 'accessoires_pour_pc': 'https://www.amazon.fr/s?i=computers&rh=n%3A93369491031%2Cp_72%3A437873031&dc&fs=true&qid=1705003327&rnid=437872031&ref=sr_nr_p_72_1&ds=v1%3AF2smD7hNdVgZQQF8cTOQWH9w5xtTQghE83O0%2Bzuef2w', 'claviers': 'https://www.amazon.fr/s?i=specialty-aps&rh=n%3A21734146031%2Cp_72%3A437873031&dc&fs=true&qid=1705003328&rnid=437872031&ref=sr_nr_p_72_1&ds=v1%3AStYEa8QmOAjeTY8mZdKrjzH3bIq%2F1T6BYmBPZQzzdRo', 'souris': 'https://www.amazon.fr/s?i=specialty-aps&rh=n%3A21734152031%2Cp_72%3A437873031&dc&fs=true&qid=1705003329&rnid=437872031&ref=sr_nr_p_72_1&ds=v1%3ACT7mK94SH1cKBzPA%2FV31kbMgb7qyX2niaC03plQXSnY', 'serveurs': 'https://www.amazon.fr/s?keywords=Serveurs&i=computers&rh=n%3A429881031%2Cp_72%3A437873031&dc&c=ts&qid=1705003331&rnid=437872031&ts_id=429881031&ref=sr_nr_p_72_1&ds=v1%3AolzhMAk6erSRD%2FjSwWMmJL6IciI%2FJ4r8Acq0cohOHEw', 'casque': 'https://www.amazon.fr/s?i=specialty-aps&srs=21734147031&rh=n%3A21734147031%2Cp_72%3A437873031&dc&fs=true&qid=1705003332&rnid=437872031&ref=sr_nr_p_72_1&ds=v1%3ARVv9sXjfzzl6wGPcdoOih7zHmTdin8jXDJgF0oCH0Mc', 'reseaux': 'https://www.amazon.fr/s?i=computers&rh=n%3A427939031%2Cp_72%3A437873031&dc&fs=true&qid=1705003334&rnid=437872031&ref=sr_nr_p_72_1&ds=v1%3Anod51V76mh%2FowXoDCeB2QWFpfv7MCuxEFBqGdpBYmuU', 'high_tech': 'https://www.amazon.fr/s?i=electronics&rh=n%3A13921051%2Cp_72%3A437873031&dc&fs=true&qid=1705003345&rnid=437872031&ref=sr_nr_p_72_1&ds=v1%3ADy0gvuzqKffXa8d%2FC0e0K8Y4bP%2Fo5fmGd993Bis7bhA'}

In [ ]:
for categorie in mon_shop:
  print(mon_shop[categorie])


https://www.amazon.fr/s?i=computers&rh=n%3A429879031%2Cp_72%3A437873031&dc&fs=true&qid=1705003247&rnid=437872031&ref=sr_nr_p_72_1&ds=v1%3AZjUdxXdQvb2Nq91AfqrfymHwIm3%2BGotg3kJTB1IxlCQ
https://www.amazon.fr/s?i=specialty-aps&rh=n%3A21734133031%2Cp_72%3A437873031&dc&fs=true&qid=1705003254&rnid=437872031&ref=sr_nr_p_72_1&ds=v1%3AvnKhp6DnaVRdCUou%2BP0VZxjkRqyIAGXy%2B9ZyJoFxmuA
https://www.amazon.fr/s?i=computers&rh=n%3A429882031%2Cp_72%3A437873031&dc&fs=true&qid=1705003255&rnid=437872031&ref=sr_nr_p_72_1&ds=v1%3AAAtXGEKVNYLOWrN3n%2BsV1JVHK3IGRXmiTagBAm5LDag
https://www.amazon.fr/s?i=computers&rh=n%3A427942031%2Cp_72%3A437873031&dc&fs=true&qid=1705003257&rnid=437872031&ref=sr_nr_p_72_1&ds=v1%3A%2FzCDt4mP%2BoNE9DDAC%2FAvwX6XSq%2F%2BDcQ%2BVfnk5xyKZnQ
https://www.amazon.fr/s?i=computers&rh=n%3A427940031%2Cp_72%3A437873031&dc&fs=true&qid=1705003258&rnid=437872031&ref=sr_nr_p_72_1&ds=v1%3Ac%2BX%2F88QmtXyWy1G%2B%2BvDNYE%2FhyFduhjz7E27lj37rJWk
https://www.amazon.fr/s?i=computers&rh=n%3A21734136031

In [ ]:
import json
from google.colab import files

# Create a dictionary object
data = {"name": "John", "age": 30, "city": "New York"}

# Convert dictionary to JSON string
json_data = json.dumps(data)

# Save JSON string to a file
with open("data.json", "w") as f:
    f.write(json_data)

# Download JSON file to local machine
files.download("data.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
liste_notes = [1,2,3,4,5]
liste_proba = [0.04,0.03,0.1,0.11,0.72]

def probabilite(nbr_total_avis,nbr_note):
    proba = nbr_total_avis/nbr_avis
    liste_proba.append(proba)


# faire probabiite() pour chaque note

def calcul_esperance(notes, probabilites):
    esperance = sum(note * proba for note, proba in zip(notes, probabilites))
    return esperance

print(f"l'esperence est de {calcul_esperance(liste_notes,liste_proba)}")


def calcul_variance(notes, probabilites):
    variance = sum(proba*(note-calcul_esperance(notes,probabilites))**2 for note,proba in zip(notes, probabilites))
    return variance

print(f"la variance est de {calcul_variance(liste_notes,liste_proba)}")

import math

def calcul_ecart_type(notes, probabilites):
    ecart_type = math.sqrt(calcul_variance(notes, probabilites))
    return ecart_type

print(f"l'ecart type est de {calcul_ecart_type(liste_notes,liste_proba)}")


def bienayme_tchebychev(notes,probabilites):
    ecart_type = calcul_ecart_type(notes,probabilites)
    variance = calcul_variance(notes,probabilites)
    inegalite = variance/ecart_type**2
    if inegalite < 0.70:
        print(f"la Probabilité que la note ne soit pas dans l'intervalle est de {inegalite-1}% ce qui est trop faible pour etre prit en compte dans la recommendation")
    else:
        print(f"la Probabilité que la note ne soit pas dans l'intervalle est de {inegalite-1}% le prduit est donc prit en compte dans la recommendation")


bienayme_tchebychev(liste_notes,liste_proba)

